In [1]:
import os
import math

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import sklearn.gaussian_process as gp
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
filename1 = "train_predictions_model_1.txt"
filename2 = "test_predictions_model_1.txt"
df1 = pd.read_csv(filename1)
df2 = pd.read_csv(filename2)
x1 = df1[['v', 'T']].values
y1 = df1['RC'].values
X1 = np.array(x1)
Y1 = np.array(y1)
x2 = df2[['v', 'T']].values
y2 = df2['RC'].values
X2 = np.array(x2)
Y2 = np.array(y2)

temp_values1 = np.unique(X1[:, 1])
vib_values1 = np.unique(X1[:, 0])
temp_values2 = np.unique(X2[:, 1])
vib_values2 = np.unique(X2[:, 0])

print("Training v values: ", vib_values1)
print("Testing v values: ", vib_values2)

if temp_values1  != temp_values2:
    print("ERROR Temperature values are different between training and testing datasets.")    
    print("Testing T values: ", temp_values2)
    print("Training T values: ", temp_values1)


fig = plt.figure()
ax = fig.add_subplot(projection='3d')
xdim1 = len(temp_values1)
ydim1 = len(vib_values1)
xdim2 = len(temp_values2)
ydim2 = len(vib_values2)
for xidx in range(xdim1):
    t = temp_values1[xidx]
    for yidx in range(ydim1):
        v =  vib_values1[yidx]
        x = float(t)
        y = float(v)
        z = df1[(df1['v'] == v) & (df1['T'] == t)]['RC'].values[0]
        ax.scatter(x, y, z, marker="o", color="b")

for xidx in range(xdim2):
    t = temp_values2[xidx]
    for yidx in range(ydim2):
        v =  vib_values2[yidx]
        x = float(t)
        y = float(v)
        z = df2[(df2['v'] == v) & (df2['T'] == t)]['RC'].values[0]
        ax.scatter(x, y, z, marker="o", color="r")  

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.gcf().set_size_inches(20, 15)

plt.show()